<a href="https://colab.research.google.com/github/06-11-2002/06-11-2002/blob/main/importing_the_necessary_Python_libraries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense



In [ ]:
with open('/content/sharelock.txt', 'r', encoding='utf-8') as file:
    text = file.read()


//Now let’s tokenize the text to create a sequence of words:

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1


//Now let’s create input-output pairs by splitting the text into sequences of tokens and forming n-grams from the sequences:

In [ ]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


In [ ]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))


In [ ]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]



In [ ]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))



In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 16, 100)           846400    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8464)              1278064   
                                                                 
Total params: 2275064 (8.68 MB)
Trainable params: 2275064 (8.68 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=10, verbose=1)


Epoch 1/10
3084/3084 [==============================] - 230s 74ms/step - loss: 6.2819 - accuracy: 0.0752
Epoch 2/10
3084/3084 [==============================] - 223s 72ms/step - loss: 5.5398 - accuracy: 0.1249
Epoch 3/10
2163/3084 [====================>.........] - ETA: 1:04 - loss: 5.1394 - accuracy: 0.1475

In [ ]:
seed_text = "i am Yasmin"
next_words = 2

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

